# Complete Fuzzing Pipeline Analysis

Comprehensive analysis from patch arrival to test execution.

## Pipeline Overview:

### STATIC ANALYSIS (Host)
1. Patch Loading
2. Repository Setup  
3. Static Analysis (Pylint, Flake8, Radon, Mypy, Bandit)

### DYNAMIC ANALYSIS (Container)
4. Build Singularity Container
5. Install Dependencies
6. Run Existing Tests
7. Patch Analysis
8. Generate Hypothesis Tests
9. Execute Tests
10. Coverage Analysis
11. Final Verdict

## Imports

In [1]:
import sys, subprocess
from pathlib import Path
import json, time, ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path.cwd()
sys.path.append(str(PROJECT_ROOT))

from swebench_integration import DatasetLoader, PatchLoader
from verifier.dynamic_analyzers.patch_analyzer import PatchAnalyzer
from verifier.dynamic_analyzers.test_generator import HypothesisTestGenerator
from verifier.dynamic_analyzers.singularity_executor import SingularityTestExecutor
from verifier.dynamic_analyzers.coverage_analyzer import CoverageAnalyzer
from verifier.dynamic_analyzers.test_patch_singularity import build_singularity_image, install_package_in_singularity, run_tests_in_singularity
import streamlit.modules.static_eval.static_modules.code_quality as code_quality
import streamlit.modules.static_eval.static_modules.syntax_structure as syntax_structure
from verifier.utils.diff_utils import parse_unified_diff, filter_paths_to_py

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
print("✓ Imports OK")

/fs/nexus-scratch/ihbas/miniconda3/envs/verifier_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Imports OK


---
# STATIC ANALYSIS
---

## Stage 1: Load Patch

In [2]:
REPO_FILTER = "pytest-dev/pytest"

loader = DatasetLoader("princeton-nlp/SWE-bench_Verified", hf_mode=True, split="test")
sample = next(loader.iter_samples(limit=1, filter_repo=REPO_FILTER), None)

if sample:
    print(f"✓ {sample.get('metadata', {}).get('instance_id', 'unknown')}")
    print(f"  Repo: {sample['repo']}")
    print(f"\nPatch preview:\n{sample['patch'][:400]}...")
else:
    raise Exception(f"No sample found for {REPO_FILTER}")

✓ pytest-dev__pytest-10051
  Repo: pytest-dev/pytest

Patch preview:
diff --git a/src/_pytest/logging.py b/src/_pytest/logging.py
--- a/src/_pytest/logging.py
+++ b/src/_pytest/logging.py
@@ -40,7 +40,6 @@
 else:
     logging_StreamHandler = logging.StreamHandler
 
-
 DEFAULT_LOG_FORMAT = "%(levelname)-8s %(name)s:%(filename)s:%(lineno)d %(message)s"
 DEFAULT_LOG_DATE_FORMAT = "%H:%M:%S"
 _ANSI_ESCAPE_SEQ = re.compile(r"\x1b\[[\d;]+m")
@@ -345,6 +344,10 @@ def rese...


## Stage 2: Setup Repository


In [3]:
patcher = PatchLoader(sample=sample, repos_root="./repos_temp")
repo_path = patcher.clone_repository()
patch_result = patcher.apply_patch()

print(f"✓ Repo: {repo_path}")
print(f"✓ Patch: {'Applied' if patch_result['applied'] else 'FAILED'}")

[+] Cloning pytest-dev/pytest into /fs/nexus-scratch/ihbas/verifier_harness/repos_temp/pytest-dev__pytest ...
✓ Repo: /fs/nexus-scratch/ihbas/verifier_harness/repos_temp/pytest-dev__pytest
✓ Patch: Applied


## Stage 2b: Apply Test Patch (if exists)

In [4]:
# In SWE-bench, test_patch contains additional tests needed to validate the fix
# These tests (like test_clear_for_call_stage) don't exist until we apply the test_patch
test_patch = sample.get('metadata', {}).get('test_patch', '')

if test_patch and test_patch.strip():
    print("📝 Applying test_patch...")
    try:
        test_patch_result = patcher.apply_additional_patch(test_patch)
        print(f"✓ Test patch applied: {test_patch_result.get('log', 'success')}")
    except Exception as e:
        print(f"⚠️ Test patch application failed: {e}")
else:
    print("ℹ️  No test_patch in metadata (tests already exist in repo)")

📝 Applying test_patch...
✓ Test patch applied: Additional patch applied successfully.


## Stage 3: Static Analysis

In [5]:
config = {
    'checks': {'pylint': True, 'flake8': True, 'radon': True, 'mypy': True, 'bandit': True},
    'weights': {'pylint': 0.5, 'flake8': 0.15, 'radon': 0.25, 'mypy': 0.05, 'bandit': 0.05}
}

print("🔍 Static analysis...")
cq_results = code_quality.analyze(str(repo_path), sample['patch'], config)
ss_results = syntax_structure.run_syntax_structure_analysis(str(repo_path), sample['patch'])

sqi_data = cq_results.get('sqi', {})
print(f"✓ SQI: {sqi_data.get('SQI', 0)}/100 ({sqi_data.get('classification', 'Unknown')})")

🔍 Static analysis...
✓ SQI: 74.67/100 (Good)


---
# DYNAMIC ANALYSIS (Container)
---

## Stage 4: Build Container

In [14]:
CONTAINER_IMAGE_PATH = "/fs/nexus-scratch/ihbas/.containers/singularity/verifier-swebench.sif"
PYTHON_VERSION = "3.11"

print("🐳 Building container...")
image_path = build_singularity_image(CONTAINER_IMAGE_PATH, PYTHON_VERSION, force_rebuild=False)

result = subprocess.run(
    ["singularity", "exec", str(image_path), "python", "--version"],
    capture_output=True, text=True
)
print(f"✓ Container: {image_path}")
print(f"  {result.stdout.strip()}")

🐳 Building container...
✅ Singularity image already exists: /fs/nexus-scratch/ihbas/.containers/singularity/verifier-swebench.sif
✓ Container: /fs/nexus-scratch/ihbas/.containers/singularity/verifier-swebench.sif
  Python 3.11.14


## Stage 5: Install Dependencies

In [15]:
print("📦 Installing dependencies...")

install_result = install_package_in_singularity(
    repo_path=Path(repo_path),
    image_path=CONTAINER_IMAGE_PATH
)

if install_result.get("installed"):
    print("✓ Dependencies installed")
elif install_result.get("returncode") != 0:
    print(f"⚠️ Install issues (code {install_result.get('returncode')})")
    print(install_result.get('stderr', '')[-500:])
else:
    print("⚠️ No setup.py/pyproject.toml")

📦 Installing dependencies...
📦 Package structure detected in: /fs/nexus-scratch/ihbas/verifier_harness/repos_temp/pytest-dev__pytest
   Setup files found: setup.py=True, pyproject.toml=True, setup.cfg=True
   Package will be accessible via PYTHONPATH=/workspace during test execution
✓ Dependencies installed


## Stage 6: Run Existing Tests

In [16]:
print("🧪 Running existing tests...\n")

# Get tests from metadata
fail_to_pass = sample.get('metadata', {}).get('FAIL_TO_PASS', '[]')
pass_to_pass = sample.get('metadata', {}).get('PASS_TO_PASS', '[]')

print(f"  FAIL_TO_PASS: {fail_to_pass}")
print(f"  PASS_TO_PASS: {pass_to_pass}\n")

# Parse test lists
try:
    f2p = ast.literal_eval(fail_to_pass) if isinstance(fail_to_pass, str) else fail_to_pass
    p2p = ast.literal_eval(pass_to_pass) if isinstance(pass_to_pass, str) else pass_to_pass
except:
    f2p, p2p = [], []

all_tests = f2p + p2p

# Use the proper function from test_patch_singularity
test_result = run_tests_in_singularity(
    repo_path=Path(repo_path),
    tests=all_tests,
    image_path=CONTAINER_IMAGE_PATH
)

print(f"Exit: {test_result['returncode']}")
print((test_result['stdout'] + test_result['stderr'])[-1500:])
print(f"\n{'✓' if test_result['returncode'] == 0 else '⚠️'} Tests {'passed' if test_result['returncode'] == 0 else 'had issues'}")

🧪 Running existing tests...

  FAIL_TO_PASS: ["testing/logging/test_fixture.py::test_clear_for_call_stage"]
  PASS_TO_PASS: ["testing/logging/test_fixture.py::test_change_level", "testing/logging/test_fixture.py::test_with_statement", "testing/logging/test_fixture.py::test_log_access", "testing/logging/test_fixture.py::test_messages", "testing/logging/test_fixture.py::test_record_tuples", "testing/logging/test_fixture.py::test_unicode", "testing/logging/test_fixture.py::test_clear", "testing/logging/test_fixture.py::test_caplog_captures_for_all_stages", "testing/logging/test_fixture.py::test_fixture_help", "testing/logging/test_fixture.py::test_change_level_undo", "testing/logging/test_fixture.py::test_change_level_undos_handler_level", "testing/logging/test_fixture.py::test_ini_controls_global_log_level", "testing/logging/test_fixture.py::test_caplog_can_override_global_log_level", "testing/logging/test_fixture.py::test_caplog_captures_despite_exception", "testing/logging/test_fixture

## Stage 7: Analyze Patch

In [17]:
print("🔍 Analyzing patch...")

patch_analyzer = PatchAnalyzer()
modified_files = filter_paths_to_py(list(parse_unified_diff(sample['patch']).keys()))

if modified_files:
    first_file_path = modified_files[0]  # e.g., "src/_pytest/logging.py"
    first_file = Path(repo_path) / first_file_path
    patched_code = first_file.read_text(encoding='utf-8')
    
    # Pass file_path to parse_patch for proper module detection
    patch_analysis = patch_analyzer.parse_patch(sample['patch'], patched_code, file_path=first_file_path)
    
    print(f"✓ Files: {len(modified_files)}")
    print(f"  Module: {patch_analysis.module_path}")
    print(f"  Functions: {patch_analysis.changed_functions}")
    if patch_analysis.class_context:
        print(f"  Classes: {list(patch_analysis.class_context.values())}")
    print(f"  Lines: {len(patch_analysis.all_changed_lines)}")
else:
    patch_analysis = None
    patched_code = None

🔍 Analyzing patch...
✓ Files: 1
  Module: _pytest.logging
  Functions: ['clear', 'clear']
  Classes: ['LogCaptureFixture']
  Lines: 5


## Stage 8: Generate Tests

In [18]:
if patch_analysis and patch_analysis.changed_functions:
    print("🧬 Generating tests...")
    
    test_generator = HypothesisTestGenerator()
    test_code = test_generator.generate_tests(patch_analysis, patched_code)
    test_count = test_code.count('def test_')
    
    print(f"✓ Generated {test_count} tests")
    print(f"\n{test_code[:600]}...")
else:
    test_code = None
    test_count = 0

🧬 Generating tests...
✓ Generated 2 tests

# Auto-generated change-aware fuzzing tests for patch validation
import pytest
from hypothesis import given, strategies as st, settings
from hypothesis import assume
import sys
from pathlib import Path

# Import from patched module: _pytest.logging
from _pytest.logging import LogCaptureFixture

def test_clear_exists():
    """Verify LogCaptureFixture.clear exists and is callable"""
    assert hasattr(LogCaptureFixture, 'clear'), 'LogCaptureFixture should have clear method'
    # Note: Full property-based testing of methods requires instance creation
    # which is complex without knowing const...


## Stage 9: Execute Tests

In [19]:
if test_code:
    print("🐳 Executing change-aware fuzzing tests...\n")
    
    executor = SingularityTestExecutor(CONTAINER_IMAGE_PATH, timeout=120)
    start = time.time()
    
    try:
        # Pass module_name from patch_analysis for proper coverage tracking
        module_name = patch_analysis.module_path if patch_analysis else None
        
        success, output, coverage_data = executor.run_tests_with_existing_infrastructure(
            Path(repo_path), 
            test_code,
            module_name=module_name
        )
        
        elapsed = time.time() - start
        print(f"{'✓ PASSED' if success else '❌ FAILED'} ({elapsed:.1f}s)\n")
        
        # Show full output to see actual test results
        print("=== FULL OUTPUT ===")
        print(output)
        print("=== END OUTPUT ===\n")
        
        # Also show just the test summary
        if "passed" in output or "PASSED" in output:
            lines = output.split('\n')
            for i, line in enumerate(lines):
                if 'test_' in line or 'passed' in line.lower() or 'failed' in line.lower():
                    print(line)
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        success = False
        coverage_data = {}
else:
    success = True
    coverage_data = {}

🐳 Executing change-aware fuzzing tests...

✓ PASSED (7.0s)

=== FULL OUTPUT ===
============================= test session starts ==============================
platform linux -- Python 3.11.14, pytest-9.0.1, pluggy-1.6.0 -- /usr/local/bin/python3.11
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: /workspace
configfile: pyproject.toml
plugins: hypothesis-6.148.1, cov-7.0.0, timeout-2.4.0, xdist-3.8.0
timeout: 120.0s
timeout method: signal
timeout func_only: False
collecting ... collected 0 items

============================ no tests ran in 1.84s =============================
ERROR: file or directory not found: test_generated.py

============================= test session starts ==============================
platform linux -- Python 3.11.14, pytest-9.0.1, pluggy-1.6.0 -- /usr/local/bin/python3.11
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: /workspace
configfile: pyproject.toml
plugins: hypothesis-6.148.1, cov-7.0.0, timeout-2.4.0, xdist-3.8.0
timeout: 1

## Stage 10: Coverage

In [12]:
if patch_analysis and coverage_data:
    coverage_analyzer = CoverageAnalyzer()
    coverage_result = coverage_analyzer.calculate_changed_line_coverage(
        coverage_data, patch_analysis.changed_lines, patch_analysis.all_changed_lines
    )
    print(f"📊 Coverage: {coverage_result['overall_coverage']:.1%}")
    print(f"   {coverage_result['total_covered_lines']}/{coverage_result['total_changed_lines']} lines")
else:
    coverage_result = {'overall_coverage': 0.0, 'total_changed_lines': 0, 'total_covered_lines': 0}

## Stage 11: Verdict

In [13]:
sqi_score = sqi_data.get('SQI', 0) / 100.0
coverage_score = coverage_result.get('overall_coverage', 0.0)

if sqi_score < 0.5:
    verdict = 'REJECT'
    reason = f'Poor SQI ({sqi_score:.2f})'
elif not success:
    verdict = 'REJECT'
    reason = 'Tests failed'
elif coverage_score < 0.5:
    verdict = 'WARNING'
    reason = f'Low coverage ({coverage_score:.1%})'
else:
    verdict = 'ACCEPT'
    reason = 'All checks passed'

print("\n" + "="*80)
print("VERDICT")
print("="*80)
print(f"{verdict}: {reason}")
print(f"\nSQI: {sqi_score:.2%} | Tests: {test_count} | Coverage: {coverage_score:.1%}")
print("="*80)


VERDICT

SQI: 74.67% | Tests: 2 | Coverage: 0.0%
